In [1]:
import sys
sys.path.append('/Users/apple/Desktop/AiCore Bootcamp/multinational-retail-data-centralisation939')

In [2]:
from src.database_utils import DatabaseConnector
from src.data_cleaning import DataCleaning
from src.data_extraction import DataExtractor
from src.data_extraction import header
import numpy as np
import pandas as pd

In [3]:
# Calculate number of stores
number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
extractor = DataExtractor()
number_of_stores = extractor.list_number_of_stores(number_of_stores_endpoint,header)

Number of stores: 451


In [4]:
# Extract store data
store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'

store_data = extractor.retrieve_stores_data(store_endpoint, number_of_stores, header)

In [5]:
store_data.head()

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [6]:
# Missing values before cleaning
missing_values_per_column = store_data.isnull().sum()
print(missing_values_per_column)

index              0
address            0
longitude          0
lat              440
locality           0
store_code         0
staff_numbers      0
opening_date       0
store_type         0
latitude           1
country_code       0
continent          0
dtype: int64


In [7]:
# Clean data
cleaner = DataCleaning()
cleaned_store_data = cleaner.clean_store_data(store_data)

In [8]:
cleaned_store_data.head()

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,NaN,<NA>,N/A,WEB-1388012W,325,2010-06-12,Web Portal,<NA>,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [9]:
# Missing values after cleaning
missing_values_per_column = cleaned_store_data.isnull().sum()
print(missing_values_per_column)

address          9
longitude        9
locality         8
store_code       8
staff_numbers    0
opening_date     8
store_type       8
latitude         9
country_code     8
continent        8
dtype: int64


In [10]:
# Upload to SQL
instance = DatabaseConnector('../../sales_data_creds.yaml')
instance.upload_to_db(cleaned_store_data,'dim_store_details')

449